# 🏇 UNITY-DUAL ENHANCED CD SYSTEM v3.1

Enhanced Course/Distance weighting (Fontwell evidence)

Focus: **Top 3 finishers** (not just winners)

**Research and educational purposes only**


In [ ]:
# Step 1 — Imports
from typing import List, Dict, Optional, Tuple
import re


In [ ]:
# Step 2 — Full class definition
class UnityDualEnhancedCD:
    def __init__(self):
        \"\"\"Enhanced system maintaining trainer ratings but boosting CD markers\"\"\"
        self.trainer_ratings = {
            # Elite tier
            'Paul Nicholls': 4.2, 'P F Nicholls': 4.2,
            'Dan Skelton': 4.0, 'D Skelton': 4.0,
            'Nicky Henderson': 3.8,
            'Willie Mullins': 4.2,
            'Gordon Elliott': 4.0,
            'Philip Hobbs': 3.4, 'P Hobbs': 3.4,
            
            # Strong tier
            'Mickey Bowen': 2.2, 'mickey bowen': 2.2,
            'N P Mulholland': 2.2,
            'Evan Williams': 2.4,
            'W Greatrex': 2.6,
            'C E Longsdon': 2.4,
            'Jamie Snowden': 3.0,
            'Jonjo O\'Neill': 2.8, 'Jonjo & A J O\'Neill': 2.8,
            
            # Competent tier
            'G & J Moore': 2.2,
            'C Gordon': 2.0,
            'Dr R Newland': 2.4, 'dr r newland': 2.4,
            'J Insole': 1.6,
            'Christian Williams': 2.4,
            'James Owen': 2.0,
            'B Pauling': 2.2,
            'Miss E C Lavelle': 2.2,
            'David Dennis': 1.8,
            'J Tizzard': 2.6
        }
        
        # ENHANCED CD multipliers based on Fontwell evidence
        self.cd_multipliers = {
            '2m': {
                'CD': 2.2,
                'C_or_D': 1.6
            },
            '2m_4f': {
                'CD': 3.0,
                'C_or_D': 2.2
            },
            '2m_7f': {
                'CD': 3.6,
                'C_or_D': 2.6
            },
            '3m+': {
                'CD': 4.2,
                'C_or_D': 3.0
            }
        }
        
        # Adjusted distance weights for top 3 focus (less win-centric)
        self.distance_weights = {
            '2m': {'rel': 1.0, 'map': 3.2, 'csi': 3.8, 'tpi': 1.6},
            '2m_4f': {'rel': 2.2, 'map': 2.4, 'csi': 2.2, 'tpi': 2.0},
            '2m_7f': {'rel': 2.6, 'map': 2.0, 'csi': 2.0, 'tpi': 2.2},
            '3m+': {'rel': 3.0, 'map': 1.8, 'csi': 1.6, 'tpi': 2.4}
        }
        
        # Lower thresholds for top 3 targeting (not just winners)
        self.top3_thresholds = {
            '2m': 7.0,
            '2m_4f': 9.5,
            '2m_7f': 11.0,
            '3m+': 12.0
        }

    def parse_form_enhanced(self, form_str: str) -> Tuple[List[int], Dict]:
        \"\"\"Enhanced form parsing focusing on consistency\"\"\"
        if not form_str or form_str.strip() == '':
            return [], {'consistency': 0, 'completion_rate': 1.0, 'recent_places': 0}
        
        positions = []
        for char in form_str.strip():
            if char.isdigit():
                pos = int(char)
                positions.append(10 if pos == 0 else pos)
            elif char.upper() in 'FURPD':
                positions.append(20)
            elif char in ['/', '-']:
                continue
        
        # Top 3 consistency metrics
        total_runs = len(positions) if positions else 1
        recent_6 = positions[-6:] if len(positions) >= 6 else positions
        places_in_recent = sum(1 for p in recent_6 if p <= 3)
        completion_rate = sum(1 for p in positions if p <= 15) / total_runs if positions else 1.0
        
        metrics = {
            'consistency': places_in_recent / len(recent_6) if recent_6 else 0,
            'completion_rate': completion_rate,
            'recent_places': places_in_recent
        }
        
        return positions, metrics

    def calculate_top3_rel(self, form: List[int], form_metrics: Dict, age: int, race_type: str) -> float:
        \"\"\"REL calculation optimized for top 3 potential\"\"\"
        if not form:
            return 1.0
        
        recent = form[-6:]
        score = 0.0
        
        # Flatter weighting for consistency over peak performance
        weights = [2.2, 2.0, 1.8, 1.6, 1.4, 1.2]
        
        for i, pos in enumerate(recent):
            if i >= len(weights):
                break
            
            weight = weights[i]
            
            # Enhanced place scoring (top 3 focused)
            if pos == 1:
                points = 6.0 * weight
            elif pos == 2:
                points = 5.0 * weight
            elif pos == 3:
                points = 4.0 * weight
            elif pos <= 6:
                points = 2.5 * weight
            elif pos <= 10:
                points = 1.5 * weight
            else:
                points = 0.2 * weight
            
            score += points
        
        # Consistency & completion bonuses
        consistency_bonus = form_metrics['consistency'] * 4.0
        completion_bonus = form_metrics['completion_rate'] * 2.5
        
        # Age adjustments for jumping
        if race_type in ['hurdle', 'chase']:
            if age in [6, 7]:
                age_mult = 1.20
            elif age in [5, 8]:
                age_mult = 1.15
            elif age >= 9:
                age_mult = 1.10
            elif age == 4:
                age_mult = 0.95
            else:
                age_mult = 1.00
        else:
            age_mult = 1.00
        
        base_rel = min(4.5, max(1.0, (score/18) + consistency_bonus + completion_bonus))
        return round(base_rel * age_mult, 2)

    def calculate_enhanced_map(self, course_markers: str, going_suit: str, race_type: str, 
                               track: str, distance_cat: str) -> float:
        \"\"\"ENHANCED MAP calculation with boosted CD weighting\"\"\"
        base_going = 2.0  # Standard assumption
        
        # ENHANCED CD BONUS
        cd_bonus = 0.0
        markers_upper = course_markers.upper() if course_markers else ''
        
        if 'CD' in markers_upper:
            cd_bonus = self.cd_multipliers[distance_cat]['CD']
        elif ('C' in markers_upper) or ('D' in markers_upper):
            cd_bonus = self.cd_multipliers[distance_cat]['C_or_D']
        
        # Race type adjustment
        type_bonus = 0.8 if race_type == 'chase' else 0.4
        
        total_map = base_going + cd_bonus + type_bonus
        return round(max(0.0, min(12.0, total_map)), 1)

    def calculate_csi(self, trainer: str, jockey: str, form: List[int], distance_cat: str) -> float:
        \"\"\"CSI calculation with distance-appropriate trainer weighting\"\"\"
        csi = 0.0
        
        # Trainer scoring
        trainer_score = 0.0
        trainer_lower = (trainer or '').lower().strip()
        
        for trainer_name, rating in self.trainer_ratings.items():
            if trainer_name.lower() in trainer_lower:
                trainer_score = rating
                break
        
        # Distance multipliers for trainer impact
        if distance_cat == '2m':
            trainer_mult = 1.6
        elif distance_cat == '2m_4f':
            trainer_mult = 1.2
        elif distance_cat == '2m_7f':
            trainer_mult = 1.0
        else:  # 3m+
            trainer_mult = 0.9
        
        csi += trainer_score * trainer_mult
        
        # Top jockey bonus (small)
        top_jockeys = ['harry cobden', 'brian hughes', 'sean bowen', 'harry skelton', 
                       'james bowen', 'david bass', 'nico de boinville']
        
        jockey_lower = (jockey or '').lower()
        for top_jockey in top_jockeys:
            if top_jockey in jockey_lower:
                csi += 1.0
                break
        
        # Form-based connection bonus
        if form:
            recent = form[-3:] if len(form) >= 3 else form
            count = len(recent) if len(recent) > 0 else 1
            recent_avg = sum(p for p in recent if p <= 15) / count
            if recent_avg <= 3.0:
                csi += 2.0
            elif recent_avg <= 5.0:
                csi += 1.2
        
        return round(min(csi, 10.0), 1)

    def calculate_tpi(self, or_rating: int, course_markers: str, race_type: str, 
                      distance_cat: str, race_class: int = 4) -> int:
        \"\"\"TPI calculation with maintained class emphasis\"\"\"
        tpi = 0
        
        # OR rating tiers
        if or_rating is None:
            base_or = 0
        elif or_rating >= 130:
            base_or = 4
        elif or_rating >= 110:
            base_or = 3
        elif or_rating >= 90:
            base_or = 2
        elif or_rating >= 70:
            base_or = 1
        else:
            base_or = 0
        
        # Distance adjustment for class importance
        if distance_cat in ['2m_7f', '3m+']:
            tpi += base_or * 2
        else:
            tpi += base_or
        
        # Course markers (separate from MAP CD bonus)
        markers_upper = (course_markers or '').upper()
        if 'CD' in markers_upper:
            tpi += 2
        elif ('C' in markers_upper) or ('D' in markers_upper):
            tpi += 1
        
        # Race type and class bonuses
        if race_type == 'chase':
            tpi += 2
        elif race_type == 'hurdle':
            tpi += 1
        
        if race_class is not None and race_class >= 5:
            tpi += 1
        
        return min(tpi, 14)

    def get_distance_category(self, distance_str: str) -> str:
        \"\"\"Distance categorization\"\"\"
        distance_lower = (distance_str or '').lower()
        
        if any(d in distance_lower for d in ['2m 1f', '2m1f', '2m 2f', '2m2f', '2m']):
            return '2m'
        elif any(d in distance_lower for d in ['2m 3f', '2m3f', '2m 4f', '2m4f']):
            return '2m_4f'
        elif any(d in distance_lower for d in ['2m 5f', '2m5f', '2m 6f', '2m6f', '2m 7f', '2m7f']):
            return '2m_7f'
        elif '3m' in distance_lower:
            return '3m+'
        else:
            return '2m'

    def assess_top3_potential(self, processed_horses: List[Dict], distance_cat: str, 
                              field_size: int) -> Dict:
        \"\"\"Assessment focused on top 3 potential\"\"\"
        if not processed_horses:
            return {'status': 'REJECT', 'reason': 'No horses processed'}
        
        threshold = float(self.top3_thresholds[distance_cat])
        
        # Adjust threshold for field size
        if field_size <= 6:
            threshold *= 0.9    # Easier to place in small fields
        elif field_size >= 12:
            threshold *= 1.1    # Harder in large fields
        
        qualifying_horses = [h for h in processed_horses if h['primary_score'] >= threshold]
        
        if not qualifying_horses:
            return {
                'status': 'REJECT',
                'reason': f'No horses above {threshold:.1f} threshold',
                'top_score': processed_horses[0]['primary_score']
            }
        
        return {
            'status': 'ACCEPT',
            'qualifying_count': len(qualifying_horses),
            'threshold_used': threshold,
            'top_score': processed_horses[0]['primary_score']
        }

    def process_race_enhanced_cd(self, race_data: Dict, debug: bool = False) -> Dict:
        \"\"\"Main processing with enhanced CD weighting for top 3 potential\"\"\"
        horses = race_data.get('horses', []) or []
        field_size = len(horses)
        race_name = race_data.get('name', 'Unknown Race')
        race_time = race_data.get('time', '')
        distance_str = race_data.get('distance', '2m 1f')
        race_type = (race_data.get('race_type', 'hurdle') or 'hurdle').lower()
        track = race_data.get('track', 'default')
        race_class = race_data.get('class', 4)
        
        distance_cat = self.get_distance_category(distance_str)
        weights = self.distance_weights[distance_cat]
        
        if debug:
            print(f\"\\nENHANCED CD PROCESSING: {race_name}\")
            print(f\"Distance: {distance_str} → {distance_cat}\")
            print(f\"CD Multipliers: {self.cd_multipliers[distance_cat]}\")
        
        if not (3 <= field_size <= 20):
            return {
                'race_name': race_name,
                'race_time': race_time,
                'distance': distance_str,
                'distance_category': distance_cat,
                'field_size': field_size,
                'track': track,
                'status': 'NO BET - FIELD SIZE',
                'reason': f'Field size {field_size} outside range'
            }
        
        # Process all horses
        processed_horses = []
        
        for horse in horses:
            name = horse.get('name', 'UNKNOWN')
            form_str = horse.get('form', '')
            or_rating = horse.get('or_rating', 85)
            age = horse.get('age', 6)
            trainer = horse.get('trainer', '')
            jockey = horse.get('jockey', '')
            course_markers = horse.get('course_markers', '')
            
            # Parse form
            form, form_metrics = self.parse_form_enhanced(form_str)
            
            # Calculate metrics with ENHANCED CD weighting
            rel = self.calculate_top3_rel(form, form_metrics, age, race_type)
            map_score = self.calculate_enhanced_map(course_markers, 'good', race_type, track, distance_cat)
            csi = self.calculate_csi(trainer, jockey, form, distance_cat)
            tpi = self.calculate_tpi(or_rating, course_markers, race_type, distance_cat, race_class)
            
            # Apply distance weights
            weighted_rel = rel * weights['rel']
            weighted_map = map_score * weights['map']
            weighted_csi = csi * weights['csi']
            weighted_tpi = tpi * weights['tpi']
            
            primary_score = weighted_rel + weighted_map + weighted_csi
            total_score = primary_score + weighted_tpi
            
            processed_horses.append({
                'name': name,
                'form_str': form_str,
                'trainer': trainer,
                'jockey': jockey,
                'course_markers': course_markers,
                'age': age,
                'or_rating': or_rating,
                'raw_scores': {
                    'rel': rel,
                    'map': map_score,
                    'csi': csi,
                    'tpi': tpi
                },
                'weighted_scores': {
                    'rel': round(weighted_rel, 1),
                    'map': round(weighted_map, 1),
                    'csi': round(weighted_csi, 1),
                    'tpi': round(weighted_tpi, 1)
                },
                'primary_score': round(primary_score, 1),
                'total_score': round(total_score, 1),
                'cd_impact': round(map_score - 2.0, 1),
                'form_metrics': form_metrics
            })
            
            if debug and course_markers:
                print(f\"{name}: CD='{course_markers}' → MAP={map_score} (CD impact: +{map_score-2.0:.1f})\")
        
        # Sort by primary score
        processed_horses.sort(key=lambda x: (-x['primary_score'], -x['total_score']))
        
        # Top 3 potential assessment
        assessment = self.assess_top3_potential(processed_horses, distance_cat, field_size)
        
        if assessment['status'] == 'REJECT':
            return {
                'race_name': race_name,
                'race_time': race_time,
                'distance': distance_str,
                'distance_category': distance_cat,
                'field_size': field_size,
                'track': track,
                'status': 'NO BET - INSUFFICIENT QUALITY',
                'reason': assessment['reason'],
                'top_horse': processed_horses[0]['name'] if processed_horses else 'None',
                'top_score': assessment.get('top_score', 0)
            }
        
        # Build result for top 3 targeting
        top_selections = processed_horses[:3]
        
        return {
            'race_name': race_name,
            'race_time': race_time,
            'distance': distance_str,
            'distance_category': distance_cat,
            'field_size': field_size,
            'track': track,
            'status': 'BET - ENHANCED CD TOP 3 SYSTEM',
            'qualifying_horses': assessment['qualifying_count'],
            'threshold_used': assessment['threshold_used'],
            'cd_multipliers_used': self.cd_multipliers[distance_cat],
            'top_3_selections': [
                {
                    **horse,
                    'selection_type': f\"{i+1}{'st' if i==0 else 'nd' if i==1 else 'rd'} Choice\",
                    'top3_confidence': 'HIGH' if horse['primary_score'] >= assessment['threshold_used'] + 2 else 'MEDIUM'
                }
                for i, horse in enumerate(top_selections)
            ],
            'all_horses': processed_horses
        }

    def display_enhanced_result(self, result: Dict) -> None:
        \"\"\"Display results with CD impact highlighted\"\"\"
        print(f\"\\n{'='*70}\")
        print(\"UNITY-DUAL ENHANCED CD v3.1 - TOP 3 FOCUS\")
        print(f\"Race: {result.get('race_time','')} {result.get('track', '').upper()}\")
        print(f\"{result.get('race_name','')}\")
        print(f\"Distance: {result.get('distance','')} ({result.get('distance_category','')})\")
        print(f\"Status: {result.get('status','')}\")
        
        if str(result.get('status','')).startswith('BET'):
            print(f\"Field Size: {result['field_size']} | Qualifying: {result.get('qualifying_horses','N/A')}\")
            print(f\"Threshold Used: {result.get('threshold_used','N/A')}\")
            print(f\"CD Multipliers: {result.get('cd_multipliers_used','N/A')}\")
            
            for selection in result['top_3_selections']:
                print(f\"\\n{selection['selection_type']}: {selection['name']}\")
                print(f\"   Trainer: {selection['trainer']} | CD: '{selection['course_markers']}'\")
                print(f\"   Form: {selection['form_str']} | Age: {selection['age']} | OR: {selection['or_rating']}\")
                print(f\"   Raw: REL:{selection['raw_scores']['rel']} MAP:{selection['raw_scores']['map']} CSI:{selection['raw_scores']['csi']} TPI:{selection['raw_scores']['tpi']}\")
                print(f\"   CD Impact: +{selection['cd_impact']:.1f} | Primary Score: {selection['primary_score']} | Confidence: {selection['top3_confidence']}\")
        else:
            print(f\"Reason: {result.get('reason','')}\")
            if 'top_horse' in result:
                print(f\"Top Horse: {result['top_horse']} (Score: {result.get('top_score', 'N/A')})\")


In [ ]:
# Step 3 — Wrapper function
def run_enhanced_cd_system(race_data: Dict, debug: bool = False) -> Dict:
    """Run the Enhanced CD system for top 3 identification"""
    system = UnityDualEnhancedCD()
    result = system.process_race_enhanced_cd(race_data, debug)
    return result


In [ ]:
# Step 4 — Example usage
race_data = {
    'name': 'Fontwell Test Handicap',
    'time': '14:00',
    'distance': '2m 4f',
    'race_type': 'hurdle',
    'track': 'Fontwell',
    'class': 4,
    'horses': [
        {'name': 'MIDNIGHT JEWEL', 'form': '32121', 'or_rating': 115, 'age': 7, 'trainer': 'Fergal O\'Brien', 'jockey': 'P J Brennan', 'course_markers': 'CD'},
        {'name': 'RANDOM CHASER', 'form': '5P63', 'or_rating': 108, 'age': 9, 'trainer': 'Mickey Bowen', 'jockey': 'Sean Bowen', 'course_markers': 'C'}
    ]
}

result = run_enhanced_cd_system(race_data, debug=True)
result
